In [ ]:
# filename: sandbox_sma_implementation_comparison.ipynb

# --- Imports and Setup ---
import numpy as np
import pandas as pd
import vectorbt as vbt
from numba import njit

# --- Data Loading ---
symbol = "EURUSD=X"
end_date = pd.Timestamp.now(tz="UTC")
start_date = end_date - pd.Timedelta(days=729)
timeframe = "1h"

print(
    f"Downloading {symbol} {timeframe} data from {start_date.date()} to {end_date.date()}..."
)
try:
    ohlc_data = vbt.YFData.download(
        symbol, start=start_date, end=end_date, interval=timeframe
    ).get(["Open", "High", "Low", "Close"])

    if ohlc_data.empty:
        raise ValueError("No data returned from yfinance.")

    close_price = ohlc_data["Close"]
    print("Data download complete.\n")

except Exception as e:
    print(f"An error occurred during data download: {e}")

In [ ]:
# --- Method 1: TA-Lib ---
print("--- Running Optimization using 'TA-Lib' ---")

# 1. Create indicator factory
sma_factory_talib = vbt.IndicatorFactory.from_talib("SMA")

# 2. Define parameter ranges
fast_sma_range = np.arange(10, 30, 2)
slow_sma_range = np.arange(40, 60, 2)

# 3. Calculate indicators
fast_sma_talib = sma_factory_talib.run(
    close_price, timeperiod=fast_sma_range, short_name="fast"
)
slow_sma_talib = sma_factory_talib.run(
    close_price, timeperiod=slow_sma_range, short_name="slow"
)

# 4. Generate signals
entries = fast_sma_talib.real.vbt.crossed_above(slow_sma_talib.real)
exits = fast_sma_talib.real.vbt.crossed_below(slow_sma_talib.real)

# 5. Run portfolio backtest
portfolio_talib = vbt.Portfolio.from_signals(
    close_price, entries, exits, freq=timeframe, init_cash=10000
)

# 6. Analyze and display results
if portfolio_talib.trades.count().sum() == 0:
    print("No trades were executed for any parameter combination.")
else:
    # Find the best parameter combination
    best_params_talib = portfolio_talib.sharpe_ratio().idxmax()

    # Select the single best portfolio instance
    best_portfolio = portfolio_talib[best_params_talib]

    # Get the stats for that single portfolio
    best_stats_talib = best_portfolio.stats()

    print(f"Best Parameters (fast_period, slow_period): {best_params_talib}")
    print("Performance for Best Parameters:")
    print(
        best_stats_talib[
            ["Total Return [%]", "Sharpe Ratio", "Max Drawdown [%]", "Win Rate [%]"]
        ]
    )

    # 7. Visualize the best strategy
    # --- CORRECTED, LAYERED PLOTTING LOGIC ---

    # Get the indicator series for the best parameters
    best_fast_sma = fast_sma_talib.real[best_params_talib[0]]
    best_slow_sma = slow_sma_talib.real[best_params_talib[1]]

    # Step A: Create the base candlestick chart from the OHLC data
    fig = ohlc_data.vbt.ohlc.plot(
        title_text=f"TA-Lib SMA Strategy | Best Params: {best_params_talib}",
        template="plotly_dark",
    )

    # Step B: Add the SMA lines to the existing figure
    best_fast_sma.vbt.plot(
        fig=fig, trace_kwargs=dict(name="Fast SMA", line=dict(color="cyan"))
    )
    best_slow_sma.vbt.plot(
        fig=fig, trace_kwargs=dict(name="Slow SMA", line=dict(color="orange"))
    )

    # Step C: Add only the trade markers to the existing figure
    best_portfolio.trades.plot(fig=fig)

    # Step D: Show the final, combined figure
    fig.show()

print("-" * 50 + "\n")

In [ ]:
# --- Method 2: pandas-ta ---
print("--- Running Optimization using 'pandas-ta' ---")

# 1. Create indicator factory, naming the output 'sma'
# This tells vectorbt to create a '.sma' attribute on the output object
sma_factory_pdta = vbt.IndicatorFactory.from_pandas_ta("sma", output_names=["sma"])

# 2. Define parameter ranges (using the same as before for comparison)
fast_sma_range = np.arange(10, 30, 2)
slow_sma_range = np.arange(40, 60, 2)

# 3. Calculate indicators
# pandas-ta's period parameter is named 'length'
fast_sma_pdta = sma_factory_pdta.run(
    close_price, length=fast_sma_range, short_name="fast"
)
slow_sma_pdta = sma_factory_pdta.run(
    close_price, length=slow_sma_range, short_name="slow"
)

# 4. Generate signals using the '.sma' output attribute
entries = fast_sma_pdta.sma.vbt.crossed_above(slow_sma_pdta.sma)
exits = fast_sma_pdta.sma.vbt.crossed_below(slow_sma_pdta.sma)

# 5. Run portfolio backtest
portfolio_pdta = vbt.Portfolio.from_signals(
    close_price, entries, exits, freq=timeframe, init_cash=10000
)

# 6. Analyze and display results
if portfolio_pdta.trades.count().sum() == 0:
    print("No trades were executed for any parameter combination.")
else:
    # Find the best parameter combination
    best_params_pdta = portfolio_pdta.sharpe_ratio().idxmax()

    # Select the single best portfolio instance
    best_portfolio = portfolio_pdta[best_params_pdta]

    # Get the stats for that single portfolio
    best_stats_pdta = best_portfolio.stats()

    print(f"Best Parameters (fast_period, slow_period): {best_params_pdta}")
    print("Performance for Best Parameters:")
    print(
        best_stats_pdta[
            ["Total Return [%]", "Sharpe Ratio", "Max Drawdown [%]", "Win Rate [%]"]
        ]
    )

    # 7. Visualize the best strategy using the same layered approach

    # Get the indicator series for the best parameters
    best_fast_sma = fast_sma_pdta.sma[best_params_pdta[0]]
    best_slow_sma = slow_sma_pdta.sma[best_params_pdta[1]]

    # Step A: Create the base candlestick chart
    fig = ohlc_data.vbt.ohlc.plot(
        title_text=f"pandas-ta SMA Strategy | Best Params: {best_params_pdta}",
        template="plotly_dark",
    )

    # Step B: Add the SMA lines
    best_fast_sma.vbt.plot(
        fig=fig, trace_kwargs=dict(name="Fast SMA", line=dict(color="cyan"))
    )
    best_slow_sma.vbt.plot(
        fig=fig, trace_kwargs=dict(name="Slow SMA", line=dict(color="orange"))
    )

    # Step C: Add the trade markers
    best_portfolio.trades.plot(fig=fig)

    # Step D: Show the final figure
    fig.show()

print("-" * 50 + "\n")

In [ ]:
# --- Method 3: Custom Implementation (Numba Accelerated) ---
print("--- Running Optimization using 'Custom Numba Function' ---")

# # Import Numba and vectorbt's numba-aware functions
# import vectorbt as vbt


# CORRECTED: The custom function must handle 2D numpy arrays.
# We use vectorbt's built-in, numba-accelerated rolling_mean_nb for this.
# The @njit decorator compiles this function for massive speedup.
@njit
def custom_sma_func_nb(close, n):
    """Calculates SMA using vectorbt's numba-accelerated rolling mean."""
    # vbt.nb.rolling_mean_nb correctly handles the 2D array passed by the factory
    return vbt.nb.rolling_mean_nb(close, window=n, minp=n)


# 1. Create the indicator class from our numba-jitted function
CustomSMA = vbt.IndicatorFactory(
    input_names=["close"], param_names=["n"], output_names=["sma"]
).from_apply_func(
    custom_sma_func_nb  # Use the new, numba-aware function
)

# 2. Define parameter ranges
fast_sma_range = np.arange(10, 30, 2)
slow_sma_range = np.arange(40, 60, 2)

# 3. Calculate indicators
fast_sma_custom = CustomSMA.run(close_price, n=fast_sma_range, short_name="fast")
slow_sma_custom = CustomSMA.run(close_price, n=slow_sma_range, short_name="slow")

# 4. Generate signals
entries = fast_sma_custom.sma.vbt.crossed_above(slow_sma_custom.sma)
exits = fast_sma_custom.sma.vbt.crossed_below(slow_sma_custom.sma)

# 5. Run portfolio backtest
portfolio_custom = vbt.Portfolio.from_signals(
    close_price, entries, exits, freq=timeframe, init_cash=10000
)

# 6. Analyze and display results
if portfolio_custom.trades.count().sum() == 0:
    print("No trades were executed for any parameter combination.")
else:
    best_params_custom = portfolio_custom.sharpe_ratio().idxmax()
    best_portfolio = portfolio_custom[best_params_custom]
    best_stats_custom = best_portfolio.stats()

    print(f"Best Parameters (fast_period, slow_period): {best_params_custom}")
    print("Performance for Best Parameters:")
    print(
        best_stats_custom[
            ["Total Return [%]", "Sharpe Ratio", "Max Drawdown [%]", "Win Rate [%]"]
        ]
    )

    # 7. Visualize the best strategy
    best_fast_sma = fast_sma_custom.sma[best_params_custom[0]]
    best_slow_sma = slow_sma_custom.sma[best_params_custom[1]]

    fig = ohlc_data.vbt.ohlc.plot(
        title_text=f"Custom Numba SMA Strategy | Best Params: {best_params_custom}",
        template="plotly_dark",
    )
    best_fast_sma.vbt.plot(
        fig=fig, trace_kwargs=dict(name="Fast SMA", line=dict(color="cyan"))
    )
    best_slow_sma.vbt.plot(
        fig=fig, trace_kwargs=dict(name="Slow SMA", line=dict(color="orange"))
    )
    best_portfolio.trades.plot(fig=fig)
    fig.show()

print("-" * 50 + "\n")